In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
housing = pd.read_csv('./data/Ames_HousePrice.csv', index_col=0)
housing['PricePerGLA'] = housing['SalePrice'] / housing['GrLivArea']
pd.set_option('display.max_columns', housing.shape[1])
housing.sample(10)

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,PricePerGLA
861,531452260,1540,184000,60,RL,70.0,9109,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,RRAe,Norm,1Fam,2Story,7,5,1994,1994,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,LwQ,36.0,GLQ,596.0,122.0,754.0,GasA,Ex,Y,SBrkr,754,786,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1994.0,RFn,2.0,495.0,TA,TA,Y,140,32,0,0,0,0,NaN,NaN,NaN,0,10,2009,WD,Normal,119.480519
850,535453040,1027,125900,20,RL,80.0,8000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1962,1962,Gable,CompShg,BrkFace,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,520.0,Rec,319.0,188.0,1027.0,GasA,TA,Y,SBrkr,1027,0,0,0.0,1.0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1962.0,Unf,1.0,299.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,11,2008,WD,Normal,122.590068
389,905427030,3672,415000,75,RL,60.0,19800,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,2.5Unf,6,8,1935,1990,Gable,CompShg,BrkFace,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Rec,425.0,Unf,0.0,1411.0,1836.0,GasA,Gd,Y,SBrkr,1836,1836,0,0.0,0.0,3,1,5,1,Gd,7,Typ,2,Gd,Detchd,1993.0,Unf,2.0,836.0,TA,TA,Y,684,80,32,0,0,0,NaN,NaN,NaN,0,12,2006,WD,Normal,113.017429
12,535177100,1336,162000,20,RL,NaN,9610,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Feedr,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,632.0,TA,TA,CBlock,TA,TA,No,Rec,203.0,Unf,0.0,918.0,1121.0,GasA,Ex,Y,FuseA,1336,0,0,0.0,0.0,1,1,3,1,TA,7,Typ,1,TA,Attchd,1958.0,RFn,2.0,488.0,TA,TA,Y,80,0,0,0,0,0,NaN,NaN,NaN,0,12,2006,WD,Normal,121.257485
317,528366050,3228,430000,20,RL,NaN,12692,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,1Story,8,5,1992,1993,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,1231.0,Unf,0.0,1969.0,3200.0,GasA,Ex,Y,SBrkr,3228,0,0,1.0,0.0,3,0,4,1,Gd,10,Typ,1,Gd,Attchd,1992.0,RFn,2.0,546.0,TA,TA,Y,264,75,291,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,133.209418
134,535402270,1040,163000,20,RL,77.0,11500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,4,1967,1967,Gable,CompShg,HdBoard,HdBoard,None,0.0,Fa,TA,CBlock,TA,Fa,No,BLQ,872.0,Rec,60.0,108.0,1040.0,GasA,Gd,Y,SBrkr,1040,0,0,1.0,0.0,1,0,3,1,TA,6,Min1,1,Po,Attchd,1967.0,RFn,2.0,480.0,TA,TA,Y,0,0,156,0,0,0,NaN,MnPrv,NaN,0,5,2009,WD,Normal,156.730769
816,903454080,1376,119500,50,RM,50.0,8600,Pave,NaN,Reg,Bnk,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1.5Fin,6,6,1937,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,780.0,780.0,GasA,TA,Y,SBrkr,780,596,0,0.0,0.0,2,0,3,1,TA,7,Typ,1,Gd,Detchd,1937.0,Unf,1.0,198.0,TA,TA,N,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,86.845930
448,528178010,2200,382500,20,RL,117.0,15262,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NridgHt,Norm,Norm,1Fam,1Story,8,5,2003,2004,Hip,CompShg,VinylSd,VinylSd,BrkFace,470.0,Gd,TA,PConc,Ex,TA,Gd,GLQ,1557.0,Unf,0.0,643.0,2200.0,GasA,Ex,Y,SBrkr,2200,0,0,1.0,0.0,2,1,3,1,Ex,8,Typ,1,Gd,Attchd,2003.0,Fin,3.0,685.0,TA,TA,Y,208,55,0,0,0,0,NaN,NaN,NaN,0,7,2009,WD,Normal,173.863636
567,902104020,1416,156500,50,RM,60.0,9600,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,6,8,1900,2004,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Rec,381.0,Unf,0.0,399.0,780.0,GasA,Ex,Y,SBrkr,940,4

# Initial Cut Based On EDA

In [3]:
# Primary: (MSSubClass, HouseStyle, BldgType), MsZoning, LandContour, LotConfig, Neighborhood, Condition1, YearBuilt, 
# YearRemodAdd (binary categorical variable of 1 or 0 on remodeling), RoofStyle, (Exterior1st, Exterior2nd, 
# MasVnrType, ExterQual, ExterCond), Foundation, (BsmtQual, BsmtCond, BsmtFinType1, BsmtFinSF1, TotalBsmtSF), HeatingQC, 
# CentralAir, Electrical, (1stFlrSF, 2ndFlrSF), (BsmtFullBath, FullBath, HalfBath), (BedroomAbvGr, TotRmsAbvGrd), 
# (KitchenAbvGr, KitchenQual), (GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond,
# PavedDrive), MoSold, SaleType, SaleCondition

In [4]:
# Secondary: Alley, LotShape, LandSlope, OverallQual, OverallCond, BsmtExposure, Functional, FireplaceQu

# Features After Unary F-tests and Mutual Information

In [5]:
import sklearn
from sklearn.feature_selection import f_regression, mutual_info_regression 
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.preprocessing import LabelEncoder

In [6]:
label_encoder = LabelEncoder()
housing_temp = housing.copy()
housing_temp.drop(['PID', 'SalePrice', 'LotFrontage'], axis = 1, inplace= True)

housing_temp['YearRemodAdd'] = (housing_temp['YearBuilt'] != housing_temp['YearRemodAdd']).map(int)

features_with_NA = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 
                   'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
for feature in features_with_NA:
    housing_temp[feature].fillna('NA', inplace = True)
housing_temp = housing_temp[~housing_temp.isnull().any(axis = 1)]

features_to_be_encoded = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 
                         'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 
                         'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 
                         'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
                          'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 
                          'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 
                          'SaleCondition']
for feature in features_to_be_encoded:
    housing_temp[feature] = label_encoder.fit_transform(housing_temp[feature])

value_f, p_f = f_regression(housing_temp.iloc[:, :-1], housing_temp['PricePerGLA'])
mutual_scores = mutual_info_regression(housing_temp.iloc[:, :-1], housing_temp['PricePerGLA'])

In [7]:
p_values = pd.Series(p_f, index = housing_temp.iloc[:, :-1].columns).sort_values()
p_values.head(20)

YearBuilt       1.473189e-179
OverallQual     1.453348e-148
BsmtFinSF1      4.044464e-138
GarageYrBlt     1.974145e-136
TotalBsmtSF     1.605939e-132
ExterQual       4.187989e-129
KitchenQual     2.590570e-105
BsmtFullBath     5.960845e-96
BsmtQual         6.775011e-95
BsmtExposure     3.962220e-94
2ndFlrSF         2.204372e-93
BedroomAbvGr     1.310565e-87
GarageFinish     5.693984e-70
GarageArea       3.570340e-68
GarageCars       3.793951e-62
GarageType       8.842591e-62
1stFlrSF         1.785505e-57
BsmtFinType1     1.607023e-53
Foundation       3.105593e-51
HeatingQC        8.149750e-47
dtype: float64

In [8]:
mutual_score_values = pd.Series(mutual_scores, index = housing_temp.iloc[:,:-1].columns).sort_values(ascending = False)
mutual_score_values.head(20)

MSSubClass      0.312206
Neighborhood    0.296669
YearBuilt       0.259674
GarageYrBlt     0.206603
2ndFlrSF        0.201946
TotalBsmtSF     0.199713
HouseStyle      0.196754
BsmtQual        0.191408
BsmtFinSF1      0.189577
BsmtFinType1    0.185292
OverallQual     0.179255
ExterQual       0.141253
1stFlrSF        0.136326
BsmtExposure    0.129041
GrLivArea       0.121251
Exterior2nd     0.114470
BedroomAbvGr    0.109831
KitchenQual     0.109333
Exterior1st     0.108717
Foundation      0.106133
dtype: float64

# Features After Comparing R2 From Simple Linear Regression 

In [9]:
# Test R2 > 0.1: MsSubClass, HouseStyle, MSZoning, Neighborhood, YearBuilt, ExterQual, Foundation, BsmtQual, BsmtFinType1, 
#                BsmtFinSF1, TotalBsmtSF, BsmtFullBath, BedroomAbvGr, KitchenQual, GarageType, GarageYrBlt, GarageFinish,
#                GarageCars

# Potentials: HeatingQC, Exterior1st/Exterior2nd

In [10]:
# Test R2 > 0.15: MsSubClass, HouseStyle, Neighborhood, YearBuilt, ExterQual, Foundation, BsmtQual, BsmtFinType1, 
#                BsmtFinSF1, TotalBsmtSF, BsmtFullBath, BedroomAbvGr, KitchenQual, GarageYrBlt, GarageCars

In [11]:
# Test R2 > 0.2: MsSubClass, HouseStyle, Neighborhood, YearBuilt, ExterQual, BsmtQual, BsmtFinType1, KitchenQual,
#               GarageYrBlt

# Best Features

In [12]:
# MSSubClass, HouseStyle, Neighborhood, YearBuilt, ExterQual, Foundation, BsmtQual, BsmtFinType1, BsmtFinSF1, TotalBsmtSF, 
# BsmtFullBath, BedroomAbvGr, KitchenQual, GarageYrBlt, GarageCars

# MSSubClass

In [13]:
MSSubClass = pd.get_dummies(housing['MSSubClass'], prefix = 'MSSubClass')
MSSubClass.drop('MSSubClass_' + str(housing['MSSubClass'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('MSSubClass', axis = 1)
housing_temp = pd.concat([housing_temp, MSSubClass], axis = 1).iloc[:, -16:]
housing_temp

,PricePerGLA,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190
1,147.196262,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,132.983794,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,124.775225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,109.720885,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,136.336336,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,127.100840,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
761,72.427572,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
762,118.078176,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [14]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.388
Test R2: 0.374


# HouseStyle

In [15]:
HouseStyle = pd.get_dummies(housing['HouseStyle'], prefix = 'HouseStyle')
HouseStyle.drop('HouseStyle_' + str(housing['HouseStyle'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('HouseStyle', axis = 1)
housing_temp = pd.concat([housing_temp, HouseStyle], axis = 1).iloc[:, -8:]
housing_temp

,PricePerGLA,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
1,147.196262,0,0,0,0,0,0,0
2,132.983794,0,0,0,0,0,0,0
3,124.775225,0,0,0,0,0,0,0
4,109.720885,0,0,0,0,1,0,0
5,136.336336,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
759,127.100840,0,1,0,0,0,0,0
760,80.553953,0,0,0,0,0,0,0
761,72.427572,0,0,0,0,1,0,0
762,118.078176,0,0,0,0,1,0,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.245
Test R2: 0.237


# BldgType

In [17]:
BldgType = pd.get_dummies(housing['BldgType'], prefix = 'BldgType')
BldgType.drop('BldgType_' + str(housing['BldgType'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('BldgType', axis = 1)
housing_temp = pd.concat([housing_temp, BldgType], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE
1,147.196262,0,0,0,0
2,132.983794,0,0,0,1
3,124.775225,0,0,0,0
4,109.720885,0,0,0,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
759,127.100840,0,0,0,0
760,80.553953,0,0,0,0
761,72.427572,0,1,0,0
762,118.078176,0,0,0,0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.098
Test R2: 0.081


# MSZoning

In [19]:
MSZoning = pd.get_dummies(housing['MSZoning'], prefix = 'MSZoning')
MSZoning.drop('MSZoning_' + str(housing['MSZoning'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('MSZoning', axis = 1)
housing_temp = pd.concat([housing_temp, MSZoning], axis = 1).iloc[:, -7:]
housing_temp

,PricePerGLA,MSZoning_A (agr),MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RM
1,147.196262,0,0,0,0,0,0
2,132.983794,0,0,0,0,0,0
3,124.775225,0,1,0,0,0,0
4,109.720885,0,0,0,0,0,0
5,136.336336,0,0,0,0,0,0
...,...,...,...,...,...,...,...
759,127.100840,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,0
761,72.427572,0,0,0,0,1,0
762,118.078176,0,0,0,0,0,0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.115
Test R2: 0.105


# LandContour

In [21]:
LandContour = pd.get_dummies(housing['LandContour'], prefix = 'LandContour')
LandContour.drop('LandContour_' + str(housing['LandContour'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('LandContour', axis = 1)
housing_temp = pd.concat([housing_temp, LandContour], axis = 1).iloc[:, -4:]
housing_temp

,PricePerGLA,LandContour_Bnk,LandContour_HLS,LandContour_Low
1,147.196262,0,0,0
2,132.983794,0,0,0
3,124.775225,0,0,0
4,109.720885,0,0,0
5,136.336336,0,0,0
...,...,...,...,...
759,127.100840,0,0,0
760,80.553953,0,0,0
761,72.427572,0,1,0
762,118.078176,0,0,0


In [22]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.034
Test R2: 0.023


# LotConfig

In [23]:
LotConfig = pd.get_dummies(housing['LotConfig'], prefix = 'LotConfig')
LotConfig.drop('LotConfig_' + str(housing['LotConfig'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('LotConfig', axis = 1)
housing_temp = pd.concat([housing_temp, LotConfig], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3
1,147.196262,1,0,0,0
2,132.983794,0,0,0,0
3,124.775225,0,0,0,0
4,109.720885,1,0,0,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
759,127.100840,0,0,0,0
760,80.553953,0,1,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,0,0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.017
Test R2:-0.012


# Neighborhood

In [25]:
Neighborhood = pd.get_dummies(housing['Neighborhood'], prefix = 'Neighborhood')
Neighborhood.drop('Neighborhood_' + str(housing['Neighborhood'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('Neighborhood', axis = 1)
housing_temp = pd.concat([housing_temp, Neighborhood], axis = 1).iloc[:, -28:]
housing_temp

,PricePerGLA,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
1,147.196262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,132.983794,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,124.775225,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,109.720885,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,136.336336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,127.100840,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
761,72.427572,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
762,118.078176,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.368
Test R2: 0.393


# Condition1

In [27]:
Condition1 = pd.get_dummies(housing['Condition1'], prefix = 'Condition1')
Condition1.drop('Condition1_' + str(housing['Condition1'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('Condition1', axis = 1)
housing_temp = pd.concat([housing_temp, Condition1], axis = 1).iloc[:, -9:]
housing_temp

,PricePerGLA,Condition1_Artery,Condition1_Feedr,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn
1,147.196262,0,0,0,0,0,0,0,0
2,132.983794,0,0,0,0,0,0,0,0
3,124.775225,0,0,0,0,0,0,0,0
4,109.720885,0,0,0,0,0,0,0,0
5,136.336336,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
759,127.100840,0,0,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,0,0,0
761,72.427572,0,0,0,0,0,0,0,0
762,118.078176,0,0,0,0,0,0,0,0


In [28]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.051
Test R2: 0.012


# YearBuilt

In [29]:
housing_temp = housing[['PricePerGLA', 'YearBuilt']]
housing_temp

,PricePerGLA,YearBuilt
1,147.196262,1939
2,132.983794,1984
3,124.775225,1930
4,109.720885,1900
5,136.336336,2001
...,...,...
759,127.100840,1916
760,80.553953,1955
761,72.427572,1949
762,118.078176,2000


In [30]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.296
Test R2: 0.291


# YearRemodAdd

In [31]:
housing_temp = housing.copy()
housing_temp['Remod'] = (housing_temp['YearBuilt'] != housing_temp['YearRemodAdd']).map(int)
housing_temp = housing_temp[['PricePerGLA', 'Remod']]
housing_temp

,PricePerGLA,Remod
1,147.196262,1
2,132.983794,0
3,124.775225,1
4,109.720885,1
5,136.336336,0
...,...,...
759,127.100840,1
760,80.553953,0
761,72.427572,1
762,118.078176,0


In [32]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.027
Test R2: 0.020


# RoofStyle

In [33]:
RoofStyle = pd.get_dummies(housing['RoofStyle'], prefix = 'RoofStyle')
RoofStyle.drop('RoofStyle_' + str(housing['RoofStyle'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('RoofStyle', axis = 1)
housing_temp = pd.concat([housing_temp, RoofStyle], axis = 1).iloc[:, -6:]
housing_temp

,PricePerGLA,RoofStyle_Flat,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed
1,147.196262,0,0,0,0,0
2,132.983794,0,0,0,0,0
3,124.775225,0,0,1,0,0
4,109.720885,0,0,0,0,0
5,136.336336,0,0,0,0,0
...,...,...,...,...,...,...
759,127.100840,0,0,0,0,0
760,80.553953,0,0,1,0,0
761,72.427572,0,0,0,0,0
762,118.078176,0,0,0,0,0


In [34]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.045
Test R2: 0.027


# Exterior1st, Exterior2nd

In [35]:
housing_temp = housing.copy()
housing_temp['Exterior2nd'].replace('Brk Cmn', 'BrkComm', inplace = True)
housing_temp['Exterior2nd'].replace('CmentBd', 'CemntBd', inplace = True)
housing_temp['Exterior2nd'].replace('Wd Shng', 'WdShing', inplace= True)
housing_temp['Exterior'] = housing_temp['Exterior1st'] + '_' + housing_temp['Exterior2nd'] 
housing_temp = housing_temp[['PricePerGLA', 'Exterior']]
Exterior = pd.get_dummies(housing_temp['Exterior'], prefix = 'Exterior')
Exterior.drop('Exterior_' + str(housing_temp['Exterior'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing_temp.drop('Exterior', axis = 1)
housing_temp = pd.concat([housing_temp, Exterior], axis = 1)
housing_temp

,PricePerGLA,Exterior_AsbShng_AsbShng,Exterior_AsbShng_CemntBd,Exterior_AsbShng_Plywood,Exterior_AsbShng_Stucco,Exterior_AsbShng_Wd Sdng,Exterior_AsphShn_AsphShn,Exterior_BrkComm_BrkComm,Exterior_BrkComm_Stucco,Exterior_BrkComm_Wd Sdng,Exterior_BrkFace_AsbShng,Exterior_BrkFace_BrkFace,Exterior_BrkFace_HdBoard,Exterior_BrkFace_MetalSd,Exterior_BrkFace_Plywood,Exterior_BrkFace_Stone,Exterior_BrkFace_Stucco,Exterior_BrkFace_Wd Sdng,Exterior_CBlock_CBlock,Exterior_CBlock_VinylSd,Exterior_CemntBd_CemntBd,Exterior_CemntBd_Wd Sdng,Exterior_CemntBd_WdShing,Exterior_HdBoard_AsphShn,Exterior_HdBoard_BrkFace,Exterior_HdBoard_HdBoard,Exterior_HdBoard_ImStucc,Exterior_HdBoard_MetalSd,Exterior_HdBoard_Plywood,Exterior_HdBoard_Stucco,Exterior_HdBoard_Wd Sdng,Exterior_HdBoard_WdShing,Exterior_ImStucc_ImStucc,Exterior_MetalSd_AsphShn,Exterior_MetalSd_CBlock,Exterior_MetalSd_HdBoard,Exterior_MetalSd_MetalSd,Exterior_MetalSd_Stucco,Exterior_MetalSd_VinylSd,Exterior_MetalSd_Wd Sdng,Exterior_MetalSd_WdShing,Exterior_Plywood_BrkComm,Exterior_Plywood_CBlock,Exterior_Plywood_HdBoard,Exterior_Plywood_ImStucc,Exterior_Plywood_Plywood,Exterior_Plywood_VinylSd,Exterior_Plywood_Wd Sdng,Exterior_Plywood_WdShing,Exterior_PreCast_PreCast,Exterior_Stucco_BrkFace,Exterior_Stucco_CemntBd,Exterior_Stucco_Plywood,Exterior_Stucco_Stone,Exterior_Stucco_Stucco,Exterior_Stucco_Wd Sdng,Exterior_Stucco_WdShing,Exterior_VinylSd_AsbShng,Exterior_VinylSd_HdBoard,Exterior_VinylSd_ImStucc,Exterior_VinylSd_MetalSd,Exterior_VinylSd_Plywood,Exterior_VinylSd_Stucco,Exterior_VinylSd_Wd Sdng,Exterior_VinylSd_WdShing,Exterior_Wd Sdng_AsbShng,Exterior_Wd Sdng_BrkFace,Exterior_Wd Sdng_HdBoard,Exterior_Wd Sdng_ImStucc,Exterior_Wd Sdng_MetalSd,Exterior_Wd Sdng_Plywood,Exterior_Wd Sdng_Stucco,Exterior_Wd Sdng_VinylSd,Exterior_Wd Sdng_Wd Sdng,Exterior_Wd Sdng_WdShing,Exterior_WdShing_HdBoard,Exterior_WdShing_Plywood,Exterior_WdShing_Stucco,Exterior_WdShing_Wd Sdng,Exterior_WdShing_WdShing
1,147.196262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,132.983794,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,124.775225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,109.720885,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,136.336336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,127.100840,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
761,72.427572,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
762,118.078176,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.171
Test R2:-747402105002177593344000.000


# MasVnrType

In [37]:
MasVnrType = pd.get_dummies(housing[~housing['MasVnrType'].isnull()]['MasVnrType'], prefix = 'MasVnrType')
MasVnrType.drop('MasVnrType_' + str(housing[~housing['MasVnrType'].isnull()]['MasVnrType'].mode()[0]),
                axis = 1, inplace = True)
housing_temp = housing[~housing['MasVnrType'].isnull()].drop('MasVnrType', axis = 1)
housing_temp = pd.concat([housing_temp, MasVnrType], axis = 1).iloc[:, -4:]
housing_temp

,PricePerGLA,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_Stone
1,147.196262,0,0,0
2,132.983794,0,1,0
3,124.775225,0,0,0
4,109.720885,0,0,0
5,136.336336,0,0,0
...,...,...,...,...
759,127.100840,0,0,0
760,80.553953,0,0,0
761,72.427572,0,0,0
762,118.078176,0,1,0


In [38]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.086
Test R2: 0.100


# ExterQual

In [39]:
ExterQual = pd.get_dummies(housing['ExterQual'], prefix = 'ExterQual')
ExterQual.drop('ExterQual_' + str(housing['ExterQual'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('ExterQual', axis = 1)
housing_temp = pd.concat([housing_temp, ExterQual], axis = 1).iloc[:, -4:]
housing_temp

,PricePerGLA,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd
1,147.196262,0,0,0
2,132.983794,0,0,1
3,124.775225,0,0,1
4,109.720885,0,0,1
5,136.336336,0,0,1
...,...,...,...,...
759,127.100840,0,0,0
760,80.553953,0,0,0
761,72.427572,0,0,0
762,118.078176,0,0,1


In [40]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.248
Test R2: 0.254


# ExterCond

In [41]:
ExterCond = pd.get_dummies(housing['ExterCond'], prefix = 'ExterCond')
ExterCond.drop('ExterCond_' + str(housing['ExterCond'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('ExterCond', axis = 1)
housing_temp = pd.concat([housing_temp, ExterCond], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po
1,147.196262,0,0,0,0
2,132.983794,0,0,0,0
3,124.775225,0,0,0,0
4,109.720885,0,0,1,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
759,127.100840,0,0,0,0
760,80.553953,0,0,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,0,0


In [42]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.035
Test R2: 0.017


# Foundation

In [43]:
Foundation = pd.get_dummies(housing['Foundation'], prefix = 'Foundation')
Foundation.drop('Foundation_' + str(housing['Foundation'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('Foundation', axis = 1)
housing_temp = pd.concat([housing_temp, Foundation], axis = 1).iloc[:, -6:]
housing_temp

,PricePerGLA,Foundation_BrkTil,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood
1,147.196262,0,0,0,0,0
2,132.983794,0,0,0,0,0
3,124.775225,1,0,0,0,0
4,109.720885,1,0,0,0,0
5,136.336336,0,1,0,0,0
...,...,...,...,...,...,...
759,127.100840,1,0,0,0,0
760,80.553953,0,0,1,0,0
761,72.427572,0,0,0,0,0
762,118.078176,0,1,0,0,0


In [44]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.174
Test R2: 0.165


# BsmtQual

In [45]:
BsmtQual = pd.get_dummies(housing[~housing['BsmtQual'].isnull()]['BsmtQual'], prefix = 'BsmtQual')
BsmtQual.drop('BsmtQual_' + str(housing[~housing['BsmtQual'].isnull()]['BsmtQual'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing[~housing['BsmtQual'].isnull()].drop('BsmtQual', axis = 1)
housing_temp = pd.concat([housing_temp, BsmtQual], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Po
1,147.196262,0,0,0,0
2,132.983794,0,0,1,0
3,124.775225,0,0,0,0
4,109.720885,0,1,0,0
5,136.336336,0,0,1,0
...,...,...,...,...,...
758,113.979118,0,0,1,0
759,127.100840,0,0,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,1,0


In [46]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.283
Test R2: 0.258


# BsmtCond

In [47]:
BsmtCond = pd.get_dummies(housing[~housing['BsmtCond'].isnull()]['BsmtCond'], prefix = 'BsmtCond')
BsmtCond.drop('BsmtCond_' + str(housing[~housing['BsmtCond'].isnull()]['BsmtCond'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing[~housing['BsmtCond'].isnull()].drop('BsmtCond', axis = 1)
housing_temp = pd.concat([housing_temp, BsmtCond], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,BsmtCond_Ex,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po
1,147.196262,0,0,0,0
2,132.983794,0,0,0,0
3,124.775225,0,0,0,0
4,109.720885,0,0,0,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
758,113.979118,0,0,0,0
759,127.100840,0,0,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,0,0


In [48]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.052
Test R2: 0.041


# BsmtFinType1

In [49]:
BsmtFinType1 = pd.get_dummies(housing[~housing['BsmtFinType1'].isnull()]['BsmtFinType1'], prefix = 'BsmtFinType1')
BsmtFinType1.drop('BsmtFinType1_' + str(housing[~housing['BsmtFinType1'].isnull()]['BsmtFinType1'].mode()[0]),
                  axis = 1, inplace = True)
housing_temp = housing[~housing['BsmtFinType1'].isnull()].drop('BsmtFinType1', axis = 1)
housing_temp = pd.concat([housing_temp, BsmtFinType1], axis = 1).iloc[:, -6:]
housing_temp

,PricePerGLA,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf
1,147.196262,0,0,0,1,0
2,132.983794,0,0,0,0,0
3,124.775225,1,0,0,0,0
4,109.720885,0,0,0,0,1
5,136.336336,0,0,0,0,0
...,...,...,...,...,...,...
758,113.979118,0,0,0,0,1
759,127.100840,0,0,0,0,1
761,72.427572,0,1,0,0,0
762,118.078176,0,0,0,0,0


In [50]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.266
Test R2: 0.223


# BsmtFinSF1

In [51]:
housing_temp = housing[(~housing['BsmtFinSF1'].isnull()) & housing['BsmtFinSF1'] != 0][['PricePerGLA', 'BsmtFinSF1']]
housing_temp

,PricePerGLA,BsmtFinSF1
1,147.196262,238.0
2,132.983794,552.0
3,124.775225,737.0
5,136.336336,643.0
9,154.668166,647.0
...,...,...
755,129.213483,890.0
757,159.068627,574.0
761,72.427572,284.0
762,118.078176,841.0


In [52]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.196
Test R2: 0.160


# TotalBsmtSF

In [53]:
housing_temp = housing[(~housing['TotalBsmtSF'].isnull()) & housing['TotalBsmtSF'] != 0][['PricePerGLA', 'TotalBsmtSF']]
housing_temp

,PricePerGLA,TotalBsmtSF
1,147.196262,856.0
2,132.983794,1049.0
3,124.775225,837.0
4,109.720885,405.0
5,136.336336,810.0
...,...,...
758,113.979118,796.0
759,127.100840,952.0
761,72.427572,1001.0
762,118.078176,985.0


In [54]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.195
Test R2: 0.190


# HeatingQC

In [55]:
HeatingQC = pd.get_dummies(housing['HeatingQC'], prefix = 'HeatingQC')
HeatingQC.drop('HeatingQC_' + str(housing['HeatingQC'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('HeatingQC', axis = 1)
housing_temp = pd.concat([housing_temp, HeatingQC], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA
1,147.196262,0,0,0,1
2,132.983794,0,0,0,1
3,124.775225,0,0,0,0
4,109.720885,0,1,0,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
759,127.100840,1,0,0,0
760,80.553953,0,0,0,0
761,72.427572,0,0,0,1
762,118.078176,0,0,0,0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.111
Test R2: 0.094


# CentralAir

In [57]:
housing_temp = housing.copy()
housing_temp['CentralAir'] = (housing_temp['CentralAir'] == 'Y').map(int)
housing_temp = housing_temp[['PricePerGLA', 'CentralAir']]
housing_temp

,PricePerGLA,CentralAir
1,147.196262,1
2,132.983794,1
3,124.775225,1
4,109.720885,1
5,136.336336,1
...,...,...
759,127.100840,0
760,80.553953,1
761,72.427572,0
762,118.078176,1


In [58]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.091
Test R2: 0.044


# Electrical

In [59]:
Electrical = pd.get_dummies(housing[~housing['Electrical'].isnull()]['Electrical'], prefix = 'Electrical')
Electrical.drop('Electrical_' + str(housing[~housing['Electrical'].isnull()]['Electrical'].mode()[0]),
                axis = 1, inplace = True)
housing_temp = housing[~housing['Electrical'].isnull()].drop('Electrical', axis = 1)
housing_temp = pd.concat([housing_temp, Electrical], axis = 1).iloc[:, -4:]
housing_temp

,PricePerGLA,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP
1,147.196262,0,0,0
2,132.983794,0,0,0
3,124.775225,0,0,0
4,109.720885,0,0,0
5,136.336336,0,0,0
...,...,...,...,...
759,127.100840,0,1,0
760,80.553953,1,0,0
761,72.427572,1,0,0
762,118.078176,0,0,0


In [60]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.054
Test R2: 0.042


# 1stFlrSF

In [61]:
housing_temp = housing[['PricePerGLA', '1stFlrSF']]
housing_temp

,PricePerGLA,1stFlrSF
1,147.196262,856
2,132.983794,1049
3,124.775225,1001
4,109.720885,717
5,136.336336,810
...,...,...
759,127.100840,952
760,80.553953,1733
761,72.427572,1001
762,118.078176,985


In [62]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.075
Test R2: 0.108


# 2ndFlrSF

In [63]:
housing_temp = housing[housing['2ndFlrSF'] != 0][['PricePerGLA', '2ndFlrSF']]
housing_temp

,PricePerGLA,2ndFlrSF
4,109.720885,322
5,136.336336,855
6,103.277836,1427
12,106.750392,650
16,97.023810,794
...,...,...
753,112.530414,683
758,113.979118,918
761,72.427572,1001
762,118.078176,857


In [64]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.046
Test R2: 0.001


# BsmtFullBath

In [65]:
BsmtFullBath = pd.get_dummies(housing[~housing['BsmtFullBath'].isnull()]['BsmtFullBath'], prefix = 'BsmtFullBath')
BsmtFullBath.drop('BsmtFullBath_' + str(housing[~housing['BsmtFullBath'].isnull()]['BsmtFullBath'].mode()[0]),
                  axis = 1, inplace = True)
housing_temp = housing[~housing['BsmtFullBath'].isnull()].drop('BsmtFullBath', axis = 1)
housing_temp = pd.concat([housing_temp, BsmtFullBath], axis = 1).iloc[:, -4:]
housing_temp

,PricePerGLA,BsmtFullBath_1.0,BsmtFullBath_2.0,BsmtFullBath_3.0
1,147.196262,1,0,0
2,132.983794,1,0,0
3,124.775225,0,0,0
4,109.720885,0,0,0
5,136.336336,1,0,0
...,...,...,...,...
759,127.100840,0,0,0
760,80.553953,0,0,0
761,72.427572,0,0,0
762,118.078176,1,0,0


In [66]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.158
Test R2: 0.156


# FullBath

In [67]:
FullBath = pd.get_dummies(housing['FullBath'], prefix = 'FullBath')
FullBath.drop('FullBath_' + str(housing['FullBath'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('FullBath', axis = 1)
housing_temp = pd.concat([housing_temp, FullBath], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,FullBath_0,FullBath_1,FullBath_3,FullBath_4
1,147.196262,0,1,0,0
2,132.983794,0,0,0,0
3,124.775225,0,1,0,0
4,109.720885,0,1,0,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
759,127.100840,0,1,0,0
760,80.553953,0,0,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,0,0


In [68]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.011
Test R2:-0.023


# HalfBath

In [69]:
HalfBath = pd.get_dummies(housing['HalfBath'], prefix = 'HalfBath')
HalfBath.drop('HalfBath_' + str(housing['HalfBath'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('HalfBath', axis = 1)
housing_temp = pd.concat([housing_temp, HalfBath], axis = 1).iloc[:, -3:]
housing_temp

,PricePerGLA,HalfBath_1,HalfBath_2
1,147.196262,0,0
2,132.983794,0,0
3,124.775225,0,0
4,109.720885,0,0
5,136.336336,1,0
...,...,...,...
759,127.100840,0,0
760,80.553953,0,0
761,72.427572,0,0
762,118.078176,1,0


In [70]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.020
Test R2:-0.006


# BedroomAbvGr

In [71]:
BedroomAbvGr = pd.get_dummies(housing['BedroomAbvGr'], prefix = 'BedroomAbvGr')
BedroomAbvGr.drop('BedroomAbvGr_' + str(housing['BedroomAbvGr'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('BedroomAbvGr', axis = 1)
housing_temp = pd.concat([housing_temp, BedroomAbvGr], axis = 1).iloc[:, -7:]
housing_temp

,PricePerGLA,BedroomAbvGr_0,BedroomAbvGr_1,BedroomAbvGr_2,BedroomAbvGr_4,BedroomAbvGr_5,BedroomAbvGr_6
1,147.196262,0,0,1,0,0,0
2,132.983794,0,0,1,0,0,0
3,124.775225,0,0,1,0,0,0
4,109.720885,0,0,1,0,0,0
5,136.336336,0,0,0,0,0,0
...,...,...,...,...,...,...,...
759,127.100840,0,0,1,0,0,0
760,80.553953,0,0,0,1,0,0
761,72.427572,0,0,0,1,0,0
762,118.078176,0,0,0,0,0,0


In [72]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.140
Test R2: 0.155


# TotRmsAbvGrd

In [73]:
TotRmsAbvGrd = pd.get_dummies(housing['TotRmsAbvGrd'], prefix = 'TotRmsAbvGrd')
TotRmsAbvGrd.drop('TotRmsAbvGrd_' + str(housing['TotRmsAbvGrd'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('TotRmsAbvGrd', axis = 1)
housing_temp = pd.concat([housing_temp, TotRmsAbvGrd], axis = 1).iloc[:, -12:]
housing_temp

,PricePerGLA,TotRmsAbvGrd_2,TotRmsAbvGrd_3,TotRmsAbvGrd_4,TotRmsAbvGrd_5,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9,TotRmsAbvGrd_10,TotRmsAbvGrd_11,TotRmsAbvGrd_12,TotRmsAbvGrd_13
1,147.196262,0,0,1,0,0,0,0,0,0,0,0
2,132.983794,0,0,0,1,0,0,0,0,0,0,0
3,124.775225,0,0,0,1,0,0,0,0,0,0,0
4,109.720885,0,0,0,0,0,0,0,0,0,0,0
5,136.336336,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
759,127.100840,0,0,1,0,0,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,1,0,0,0,0,0
761,72.427572,0,0,0,0,0,1,0,0,0,0,0
762,118.078176,0,0,0,0,1,0,0,0,0,0,0


In [74]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.061
Test R2: 0.004


# KitchenAbvGr

In [75]:
KitchenAbvGr = pd.get_dummies(housing['KitchenAbvGr'], prefix = 'KitchenAbvGr')
KitchenAbvGr.drop('KitchenAbvGr_' + str(housing['KitchenAbvGr'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('KitchenAbvGr', axis = 1)
housing_temp = pd.concat([housing_temp, KitchenAbvGr], axis = 1).iloc[:, -4:]
housing_temp

,PricePerGLA,KitchenAbvGr_0,KitchenAbvGr_2,KitchenAbvGr_3
1,147.196262,0,0,0
2,132.983794,0,0,0
3,124.775225,0,0,0
4,109.720885,0,0,0
5,136.336336,0,0,0
...,...,...,...,...
759,127.100840,0,0,0
760,80.553953,0,0,0
761,72.427572,0,1,0
762,118.078176,0,0,0


In [76]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.091
Test R2: 0.054


# KitchenQual

In [77]:
KitchenQual = pd.get_dummies(housing['KitchenQual'], prefix = 'KitchenQual')
KitchenQual.drop('KitchenQual_' + str(housing['KitchenQual'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('KitchenQual', axis = 1)
housing_temp = pd.concat([housing_temp, KitchenQual], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po
1,147.196262,0,0,0,0
2,132.983794,0,0,1,0
3,124.775225,0,0,1,0
4,109.720885,0,0,0,0
5,136.336336,0,0,1,0
...,...,...,...,...,...
759,127.100840,0,1,0,0
760,80.553953,0,0,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,1,0


In [78]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.202
Test R2: 0.239


# GarageType

In [79]:
GarageType = pd.get_dummies(housing[~housing['GarageType'].isnull()]['GarageType'], prefix = 'GarageType')
GarageType.drop('GarageType_' + str(housing[~housing['GarageType'].isnull()]['GarageType'].mode()[0]),
                axis = 1, inplace = True)
housing_temp = housing[~housing['GarageType'].isnull()].drop('GarageType', axis = 1)
housing_temp = pd.concat([housing_temp, GarageType], axis = 1).iloc[:, -6:]
housing_temp

,PricePerGLA,GarageType_2Types,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd
1,147.196262,0,0,0,0,1
2,132.983794,0,0,0,0,0
3,124.775225,0,0,0,0,1
4,109.720885,0,0,0,0,1
5,136.336336,0,0,0,0,0
...,...,...,...,...,...,...
759,127.100840,0,0,0,0,1
760,80.553953,0,0,0,0,0
761,72.427572,1,0,0,0,0
762,118.078176,0,0,0,0,0


In [80]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.128
Test R2: 0.105


# GarageYrBlt

In [81]:
housing_temp = housing[(~housing['GarageYrBlt'].isnull()) & (housing['GarageYrBlt'] != 0)][['PricePerGLA', 'GarageYrBlt']]
housing_temp

,PricePerGLA,GarageYrBlt
1,147.196262,1939.0
2,132.983794,1984.0
3,124.775225,1930.0
4,109.720885,1940.0
5,136.336336,2001.0
...,...,...
759,127.100840,1916.0
760,80.553953,1955.0
761,72.427572,1949.0
762,118.078176,2000.0


In [82]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.220
Test R2: 0.234


# GarageFinish

In [83]:
GarageFinish = pd.get_dummies(housing[~housing['GarageFinish'].isnull()]['GarageFinish'], prefix = 'GarageFinish')
GarageFinish.drop('GarageFinish_' + str(housing[~housing['GarageFinish'].isnull()]['GarageFinish'].mode()[0]),
                axis = 1, inplace = True)
housing_temp = housing[~housing['GarageFinish'].isnull()].drop('GarageFinish', axis = 1)
housing_temp = pd.concat([housing_temp, GarageFinish], axis = 1).iloc[:, -3:]
housing_temp

,PricePerGLA,GarageFinish_Fin,GarageFinish_RFn
1,147.196262,0,0
2,132.983794,1,0
3,124.775225,0,0
4,109.720885,0,0
5,136.336336,1,0
...,...,...,...
759,127.100840,0,0
760,80.553953,0,0
761,72.427572,0,0
762,118.078176,1,0


In [84]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.134
Test R2: 0.112


# GarageCars

In [85]:
GarageCars = pd.get_dummies(housing[~housing['GarageCars'].isnull()]['GarageCars'], prefix = 'GarageCars')
GarageCars.drop('GarageCars_' + str(housing[~housing['GarageCars'].isnull()]['GarageCars'].mode()[0]),
                axis = 1, inplace = True)
housing_temp = housing[~housing['GarageCars'].isnull()].drop('GarageCars', axis = 1)
housing_temp = pd.concat([housing_temp, GarageCars], axis = 1).iloc[:, -6:]
housing_temp

,PricePerGLA,GarageCars_0.0,GarageCars_1.0,GarageCars_3.0,GarageCars_4.0,GarageCars_5.0
1,147.196262,0,0,0,0,0
2,132.983794,0,1,0,0,0
3,124.775225,0,1,0,0,0
4,109.720885,0,1,0,0,0
5,136.336336,0,0,0,0,0
...,...,...,...,...,...,...
759,127.100840,0,1,0,0,0
760,80.553953,0,0,0,0,0
761,72.427572,0,0,1,0,0
762,118.078176,0,0,0,0,0


In [86]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.157
Test R2: 0.177


# GarageArea

In [87]:
housing_temp = housing[(~housing['GarageArea'].isnull()) & (housing['GarageArea'] != 0)][['PricePerGLA', 'GarageArea']]
housing_temp

,PricePerGLA,GarageArea
1,147.196262,399.0
2,132.983794,266.0
3,124.775225,216.0
4,109.720885,281.0
5,136.336336,528.0
...,...,...
759,127.100840,192.0
760,80.553953,452.0
761,72.427572,871.0
762,118.078176,486.0


In [88]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.134
Test R2: 0.078


# GarageQual

In [89]:
GarageQual = pd.get_dummies(housing[~housing['GarageQual'].isnull()]['GarageQual'], prefix = 'GarageQual')
GarageQual.drop('GarageQual_' + str(housing[~housing['GarageQual'].isnull()]['GarageQual'].mode()[0]),
                axis = 1, inplace = True)
housing_temp = housing[~housing['GarageQual'].isnull()].drop('GarageQual', axis = 1)
housing_temp = pd.concat([housing_temp, GarageQual], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po
1,147.196262,0,0,0,0
2,132.983794,0,0,0,0
3,124.775225,0,0,0,0
4,109.720885,0,0,0,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
759,127.100840,0,1,0,0
760,80.553953,0,0,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,0,0


In [90]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.039
Test R2: 0.048


# GarageCond

In [91]:
GarageCond = pd.get_dummies(housing[~housing['GarageCond'].isnull()]['GarageCond'], prefix = 'GarageCond')
GarageCond.drop('GarageCond_' + str(housing[~housing['GarageCond'].isnull()]['GarageCond'].mode()[0]),
                axis = 1, inplace = True)
housing_temp = housing[~housing['GarageCond'].isnull()].drop('GarageCond', axis = 1)
housing_temp = pd.concat([housing_temp, GarageCond], axis = 1).iloc[:, -5:]
housing_temp

,PricePerGLA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po
1,147.196262,0,0,0,0
2,132.983794,0,0,0,0
3,124.775225,0,0,0,1
4,109.720885,0,0,0,0
5,136.336336,0,0,0,0
...,...,...,...,...,...
759,127.100840,0,0,0,1
760,80.553953,0,0,0,0
761,72.427572,0,0,0,0
762,118.078176,0,0,0,0


In [92]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.045
Test R2: 0.043


# PavedDrive

In [93]:
PavedDrive = pd.get_dummies(housing['PavedDrive'], prefix = 'PavedDrive')
PavedDrive.drop('PavedDrive_' + str(housing['PavedDrive'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('PavedDrive', axis = 1)
housing_temp = pd.concat([housing_temp, PavedDrive], axis = 1).iloc[:, -3:]
housing_temp

,PricePerGLA,PavedDrive_N,PavedDrive_P
1,147.196262,0,0
2,132.983794,0,0
3,124.775225,1,0
4,109.720885,1,0
5,136.336336,0,0
...,...,...,...
759,127.100840,0,1
760,80.553953,0,0
761,72.427572,0,0
762,118.078176,0,0


In [94]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.089
Test R2: 0.057


# MoSold

In [95]:
MoSold = pd.get_dummies(housing['MoSold'], prefix = 'MoSold')
MoSold.drop('MoSold_' + str(housing['MoSold'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('MoSold', axis = 1)
housing_temp = pd.concat([housing_temp, MoSold], axis = 1).iloc[:, -12:]
housing_temp

,PricePerGLA,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12
1,147.196262,0,0,1,0,0,0,0,0,0,0,0
2,132.983794,0,1,0,0,0,0,0,0,0,0,0
3,124.775225,0,0,0,0,0,0,0,0,0,1,0
4,109.720885,0,0,0,0,1,0,0,0,0,0,0
5,136.336336,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
759,127.100840,0,0,0,0,1,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,0,0,0,0,0,0
761,72.427572,0,0,0,0,0,0,1,0,0,0,0
762,118.078176,0,0,0,0,0,1,0,0,0,0,0


In [96]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.002
Test R2:-0.027


# SaleType

In [97]:
SaleType = pd.get_dummies(housing['SaleType'], prefix = 'SaleType')
SaleType.drop('SaleType_' + str(housing['SaleType'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('SaleType', axis = 1)
housing_temp = pd.concat([housing_temp, SaleType], axis = 1).iloc[:, -10:]
housing_temp

,PricePerGLA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_VWD
1,147.196262,0,0,0,0,0,0,0,0,0
2,132.983794,0,0,0,0,0,0,0,0,0
3,124.775225,0,0,0,0,0,0,0,0,0
4,109.720885,0,0,0,0,0,0,0,0,0
5,136.336336,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
759,127.100840,0,0,0,0,0,0,0,0,0
760,80.553953,0,0,0,0,0,0,0,0,0
761,72.427572,0,0,0,0,0,0,0,0,0
762,118.078176,0,0,0,0,0,0,0,0,0


In [98]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.061
Test R2: 0.052


# SaleCondition

In [99]:
SaleCondition = pd.get_dummies(housing['SaleCondition'], prefix = 'SaleCondition')
SaleCondition.drop('SaleCondition_' + str(housing['SaleCondition'].mode()[0]), axis = 1, inplace = True)
housing_temp = housing.drop('SaleCondition', axis = 1)
housing_temp = pd.concat([housing_temp, SaleCondition], axis = 1).iloc[:, -6:]
housing_temp

,PricePerGLA,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial
1,147.196262,0,0,0,0,0
2,132.983794,0,0,0,0,0
3,124.775225,0,0,0,0,0
4,109.720885,0,0,0,0,0
5,136.336336,0,0,0,0,0
...,...,...,...,...,...,...
759,127.100840,0,0,0,0,0
760,80.553953,0,0,0,0,0
761,72.427572,0,0,0,0,0
762,118.078176,0,0,0,0,0


In [100]:
X_train, X_test, y_train, y_test = train_test_split(housing_temp.iloc[:, 1:], housing_temp.iloc[:, 0],
                                                    test_size=0.3, random_state = 42)
ols = LinearRegression()
ols.fit(X_train, y_train)
print(f'Train R2:{ols.score(X_train, y_train): .3f}')
print(f'Test R2:{ols.score(X_test, y_test): .3f}')

Train R2: 0.067
Test R2: 0.034
